# Prognoza pogody za pomocą klasyfikacji Marcin Wilk

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("C:\\Users\\Lenovo\\Desktop\\Wstęp do uczenia maszynowego\\Dane\\australia.csv")

In [3]:
df.head(10)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
3,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
4,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0
5,27.1,36.1,0.0,13.0,0.0,43.0,7.0,20.0,26.0,19.0,1007.7,1007.4,8.0,8.0,30.7,34.3,0,0
6,23.3,34.0,0.0,9.8,12.6,41.0,17.0,19.0,33.0,15.0,1011.3,1009.9,3.0,1.0,25.0,31.5,0,0
7,16.1,34.2,0.0,14.6,13.2,37.0,15.0,6.0,25.0,9.0,1013.3,1009.2,1.0,1.0,20.7,32.8,0,0
8,19.0,35.5,0.0,12.0,12.3,48.0,30.0,9.0,46.0,28.0,1008.3,1004.0,1.0,5.0,23.4,33.3,0,0
9,19.7,35.5,0.0,11.0,12.7,41.0,15.0,17.0,61.0,14.0,1007.9,1005.8,1.0,5.0,24.0,33.6,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56420 entries, 0 to 56419
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MinTemp        56420 non-null  float64
 1   MaxTemp        56420 non-null  float64
 2   Rainfall       56420 non-null  float64
 3   Evaporation    56420 non-null  float64
 4   Sunshine       56420 non-null  float64
 5   WindGustSpeed  56420 non-null  float64
 6   WindSpeed9am   56420 non-null  float64
 7   WindSpeed3pm   56420 non-null  float64
 8   Humidity9am    56420 non-null  float64
 9   Humidity3pm    56420 non-null  float64
 10  Pressure9am    56420 non-null  float64
 11  Pressure3pm    56420 non-null  float64
 12  Cloud9am       56420 non-null  float64
 13  Cloud3pm       56420 non-null  float64
 14  Temp9am        56420 non-null  float64
 15  Temp3pm        56420 non-null  float64
 16  RainToday      56420 non-null  int64  
 17  RainTomorrow   56420 non-null  int64  
dtypes: flo

## Podział zbioru na testowy i treningowy

W naszym zbiorze testowym znajdzie się 60% obserwacji wybranych losowo z wejściowego zbioru, natomiast pozostałe 40% trafi do zbioru treningowego.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:,0:17], df["RainTomorrow"], test_size=0.4, random_state=0)

Sprawdźmy jeszcze jak zbalansowane są klasy w naszym zbiorze treningowym i testowym, gdyż jest to istotne w przypadku niektórych miar skuteczności klasyfikatorów.

In [6]:
Y_train.value_counts()

0    26425
1     7427
Name: RainTomorrow, dtype: int64

In [7]:
Y_test.value_counts()

0    17568
1     5000
Name: RainTomorrow, dtype: int64

Klasy nie są dobrze zbalansowane, ale zmienne o wartości 1 stanowią ponad 20%. Musimy zatem uważać w przypadku niektórych miar skuteczności.

## Regresja logistyczna

Pierwszym modelem, który dopasujemy do tych danych będzie regresja logistyczna. Mamy tu co prawda zmienne jakościowe, ale są one już zakodowane i w dodatku mają one naturalny porządek. W tym klasyfikatorze naszym hiperparametrem będzie maksymalna liczba iteracji algorytmu szukającego parametrów naszego modelu. Ustawimy go na 10000.

In [8]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=10000)

lr.fit(X_train,Y_train)
Y_hat = lr.predict(X_test)

In [9]:
table_lr=pd.crosstab(Y_test, Y_hat, rownames = ['Y_test'], colnames = ['Y_hat'])
table_lr

Y_hat,0,1
Y_test,,
0,16581,987
1,2316,2684


Jak widzimy po tabeli incydencji nasz klasyfikator dobrze radzi sobie z obserwacjami, które powinny być zerami, lecz z kolei słabo radzi sobie z obserwacjami, które powinny należeć do klasy 1. Policzmy zatem miary dopasowania naszego klasyfikatora.

In [10]:
precision_lr=table_lr.iloc[0,0]/(table_lr.iloc[0,0]+table_lr.iloc[1,0])
precision_lr

0.8774408636291475

In [11]:
recall_lr=table_lr.iloc[0,0]/(table_lr.iloc[0,0]+table_lr.iloc[0,1])
recall_lr

0.943818306010929

In [12]:
specificity_lr=table_lr.iloc[1,1]/(table_lr.iloc[1,1]+table_lr.iloc[1,0])
specificity_lr

0.5368

In [13]:
accuracy_lr=(table_lr.iloc[0,0]+table_lr.iloc[1,1])/len(Y_test)
accuracy_lr

0.853642325416519

Pomimo tego, że accuracy, recall i precision są wysokie, nie jest to raczej klasyfikator, którego chcielibyśmy używać. Ich wysokie wartości wynikają raczej z niezbalansowania klas w zbiorze testowym. Dobrą miarą dopasowania jest tutaj specifity, która wynosi lekko ponad 0.5. Oznacza, to że przewidzimy tylko połowę dni deszczowych, całą resztę klasyfikując jako te bez deszczu. 

## Naiwny Bayes

Drugim klasyfikatorem, którego użyjemy do naszego zbioru jest naiwny klasyfikator bayesowski. Jest on oparty na założeniach normalności zmiennych ciągłych i niezależności wszystkich zmiennych objaśniających, jednak intuicyjnie czujemy, że niektóre z naszych zmiennych jak temperatury o 9 i 15 będą jak najbardziej zależne. Będziemy więc nieco sceptyczni co do tego modelu, ale może okaże się, że zadziała dobrze. Hiperparametrem będzie var_smoothing czyli jaką część największej wariancji spośród wszystkich zmiennych mamy dodać do pozostałych wariancji, które bedziemy estymować w celu znalezienia parametrów rozkładu normalnego dla zmiennych ciągłych.

In [14]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB(var_smoothing=1e-10)
nb.fit(X_train,Y_train)
Y_hat = nb.predict(X_test)

In [15]:
table_nb=pd.crosstab(Y_test, Y_hat, rownames = ['Y_test'], colnames = ['Y_hat'])
table_nb

Y_hat,0,1
Y_test,,
0,14695,2873
1,1546,3454


Tabela incydencji zdecydowanie różni się od regresji logistycznej. Widzimy, że w tym przypadku gorzej klasyfikuje nam dni bezdeszczowe, jednak klasyfikuje znacznie mniej dni deszczowych jako bezdeszczowe. Sprawdźmy jak wyglądają miary jakości tego klasyfikatora

In [16]:
precision_nb=table_nb.iloc[0,0]/(table_nb.iloc[0,0]+table_nb.iloc[1,0])
precision_nb

0.9048088171910597

In [17]:
recall_nb=table_nb.iloc[0,0]/(table_nb.iloc[0,0]+table_nb.iloc[0,1])
recall_nb

0.8364640255009107

In [18]:
specificity_nb=table_nb.iloc[1,1]/(table_nb.iloc[1,1]+table_nb.iloc[1,0])
specificity_nb

0.6908

In [19]:
accuracy_nb=(table_nb.iloc[0,0]+table_nb.iloc[1,1])/len(Y_test)
accuracy_nb

0.8041917759659695

Wszystkie miary są w miarę wysokie, możemy zatem wnioskować, że brak zbalansowania klas nie miał zbytniego wpływu na nasz klasyfikator. Model ten nie wydaje się również przeuczony.

## Drzewo decyzyjne

Ostatnim klasyfikatorem, który wytestujemy bedzie drzewo decyzyjne. Powinno ono dobrze poradzić sobie ze zmiennymi jakościowymi, które są uporządkowane, jak również oczywiście ze zmiennymi binarnymi i ciągłymi. Hiperparametrami, który tutaj ustawimy bedą maksymalna liczba obserwacji w węźle przy której dalej będziemy dokonywać podziału, czyli *min_samples_split*, *max_depth* - maksymalna głębokość drzewa oraz *criterion* czyli miara, przy pomocy której będziemy wybierać zmienne i metody podziału w węzłach. 

In [20]:
from sklearn.tree import DecisionTreeClassifier,plot_tree
dt = DecisionTreeClassifier(min_samples_split=20,max_depth=5,criterion="entropy")
dt.fit(X_train,Y_train)
Y_hat = dt.predict(X_test)

In [21]:
table_dt=pd.crosstab(Y_test, Y_hat, rownames = ['Y_test'], colnames = ['Y_hat'])
table_dt

Y_hat,0,1
Y_test,,
0,16559,1009
1,2544,2456


Tabela incydencji przypomina nieco regresję logistyczną, choć jest nawet minimalnie gorsza. Powinna nam to również potwierdzić specifity, ale zobaczmy też inne miary.

In [22]:
precision_dt=table_dt.iloc[0,0]/(table_dt.iloc[0,0]+table_dt.iloc[1,0])
precision_dt

0.8668271999162436

In [23]:
recall_dt=table_dt.iloc[0,0]/(table_dt.iloc[0,0]+table_dt.iloc[0,1])
recall_dt

0.942566029143898

In [24]:
specificity_dt=table_dt.iloc[1,1]/(table_dt.iloc[1,1]+table_dt.iloc[1,0])
specificity_dt

0.4912

In [25]:
accuracy_dt=(table_dt.iloc[0,0]+table_dt.iloc[1,1])/len(Y_test)
accuracy_dt

0.842564693371145

Pomimo wysokich precision, recall i accuracy mamy raczej niską specificity, wynoszącą mniej niż 50%. Oznacza, to że nie przewidzymy ponad połowy deszczowych dni, co zdecydowanie nas nie zadowala. Pozostałe miary są wysokie najprawdopodobniej z powodu niezbalansowania klas.

## Wnioski

Pomimo tego, że założenia naiwnego Bayesa raczej nie były w większości spełnione, model ten okazał się dla mnie najlepszy. Nie miał co prawda wszystkich miar najlepszych spośród tych 3 modeli, ale wyglądał na najmniej przeuczony i najmniej wrażliwy na brak balansu w klasach. Zaletą tego modelu było jednak to, że zapewne w przeciweństwie do regresji logistycznej lepiej poradził sobie ze zmiennymi jakościowymi.

Modele regresji logistycznej i drzewa decyzyjnego raczej słabo poradziły sobie z klasyfikacją, miały niską specifity, na poziomie 0.5. Możliwe, że regresja logistyczna mogłaby sobie lepiej poradzić nawet bez zmiennych jakościowych. Po drzewie spodziewałem się trochę więcej, powinno ono poradzić sobie dobrze ze wszystkimi zmiennymi. Miało ono jednak wiele hiperparametrów, które możnaby było lepiej nastroić, więc przy tak dobranych przeze mnie może byc po prostu przeuczone.